# NBA All-Time Playoff Scoring Ranks

In [20]:
import pandas as pd
from nba_api.stats.endpoints import commonallplayers
from nba_api.stats.endpoints import playercareerstats
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import mplcursors
pd.set_option('display.max_columns', None)

In [21]:
player_data = commonallplayers.CommonAllPlayers(timeout = 30)  # API Call
player_df = player_data.common_all_players.get_data_frame().set_index('PERSON_ID') # DataFrame containing player info

In [22]:
player_df.head(10)

,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FIRST_LAST,ROSTERSTATUS,FROM_YEAR,TO_YEAR,PLAYERCODE,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,GAMES_PLAYED_FLAG,OTHERLEAGUE_EXPERIENCE_CH
PERSON_ID,,,,,,,,,,,,,
76001,"Abdelnaby, Alaa",Alaa Abdelnaby,0,1990,1994,HISTADD_alaa_abdelnaby,0,,,,,Y,00
76002,"Abdul-Aziz, Zaid",Zaid Abdul-Aziz,0,1968,1977,HISTADD_zaid_abdul-aziz,0,,,,,Y,00
76003,"Abdul-Jabbar, Kareem",Kareem Abdul-Jabbar,0,1969,1988,HISTADD_kareem_abdul-jabbar,0,,,,,Y,00
51,"Abdul-Rauf, Mahmoud",Mahmoud Abdul-Rauf,0,1990,2000,mahmoud_abdul-rauf,0,,,,,Y,00
1505,"Abdul-Wahad, Tariq",Tariq Abdul-Wahad,0,1997,2003,tariq_abdul-wahad,0,,,,,Y,00
949,"Abdur-Rahim, Shareef",Shareef Abdur-Rahim,0,1996,2007,shareef_abdur-rahim,0,,,,,Y,00
76005,"Abernethy, Tom",Tom Abernethy,0,1976,1980,HISTADD_tom_abernethy,0,,,,,Y,00
76006,"Able, Forest",Forest Able,0,1956,1956,HISTADD_frosty_able,0,,,,,Y,00
76007,"Abramovic, John",John Abramovic,0,1946,1947,HISTADD_brooms_abramovic,0,,,,,Y,00


In [23]:
# Making list 'id_list' containing PERSON_ID index from DataFrame 'player_df'
id_list = player_df.index.tolist()

In [24]:
"""
Generator 'playoff_stats' which yields the number of Playoff Games Played (GP) and Career Playoff Points (PTS)
as a list scored by every NBA player who has ever played in an NBA playoff game.
"""

def playoff_stats(player_id):

    player_stats = playercareerstats.PlayerCareerStats(player_id, timeout = 30)
    
    yield player_stats.career_totals_post_season.get_data_frame()[['GP', 'PTS']].values.tolist()

In [ ]:
stats_dict = {} # New dictionary which will contain player name and their playoff GP and PTS stats

for i in id_list: # Looping through our 'id_list' list
    
    try:
        stats_call = next(playoff_stats(i)) # Calling the generator with the player's ID as the 'player_id' variable
        
        if len(stats_call) > 0:
            stats_dict[player_df.loc[i]['DISPLAY_FIRST_LAST']] = [stats_call[0][0], stats_call[0][1]]
            # Adding their playoff GP and PTS to the 'stats_dict' dictionary
        
    except KeyError: # Not every player in NBA history has played in a playoff game.  For those who haven't, our
        continue     # generator will have nothing to yield and we will get a KeyError.  So we continue.

In [6]:
# Creating a DataFrame from our 'stats_dict' dictionary
playoff_data_DF = pd.DataFrame.from_dict(stats_dict, orient = 'index')
playoff_data_DF

,Playoff Games Played,Playoff Points Scored
Player Name,,
Alaa Abdelnaby,17,38
Zaid Abdul-Aziz,18,92
Kareem Abdul-Jabbar,237,5762
Mahmoud Abdul-Rauf,15,195
Tariq Abdul-Wahad,13,68
...,...,...
Wang Zhi-zhi,16,30
Paul Zipser,6,44
Ante Zizic,8,13


In [ ]:
# Saving our new DataFrame of all NBA players who have ever played in a playoff game as an Excel file

path = 'Playoff DataFrame.xlsx'

writer = pd.ExcelWriter(path)

playoff_data_DF.to_excel(writer, sheet_name = 'Playoff Scorer Data', index = True)

writer.save()